In [22]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

def get_all_vacancies(search_query="Аналитик данных", region=113, max_vacancies=2000):
    """Получение вакансий по ключевому слову с поддержкой нескольких регионов"""
    base_url = "https://api.hh.ru/vacancies"
    all_vacancies = []
    page = 0
    per_page = 100

    # Если регион передан как список, преобразуем в строку
    area_ids = region if isinstance(region, list) else [region]

    print(f"🔍 Поиск вакансий по запросу: '{search_query}'")

    for area_id in area_ids:
        print(f"🌍 Регион: {get_region_name(area_id)} (ID: {area_id})")
        page = 0  # Сбрасываем счетчик страниц для нового региона

        while True:
            params = {
                "text": search_query,
                "area": area_id,
                "page": page,
                "per_page": per_page,
                "search_field": "name"  # Ищем в названии вакансии
            }

            try:
                response = requests.get(base_url, params=params)
                response.raise_for_status()
                data = response.json()

                if not data.get("items"):
                    print(f"  ➤ Вакансии в регионе закончились")
                    break

                all_vacancies.extend(data["items"])
                print(f"  ➤ Страница {page+1}: +{len(data['items'])} вакансий (всего: {len(all_vacancies)})")

                # Проверка достижения лимита
                if len(all_vacancies) >= max_vacancies or page >= data["pages"] - 1:
                    break

                page += 1
                time.sleep(0.2)

            except Exception as e:
                print(f"⚠️ Ошибка: {e}")
                break

            if len(all_vacancies) >= max_vacancies:
                print(f"🚩 Достигнут лимит в {max_vacancies} вакансий")
                break

    return all_vacancies[:max_vacancies]

def get_region_name(region_id):
    """Получение названия региона по ID"""
    try:
        response = requests.get(f"https://api.hh.ru/areas/{region_id}")
        return response.json()["name"]
    except:
        return f"Неизвестный регион ({region_id})"

def clean_html(html_text):
    """Очистка HTML-форматирования из текста"""
    if not html_text:
        return ""
    soup = BeautifulSoup(html_text, "html.parser")
    return soup.get_text(separator=" ", strip=True)

def parse_vacancies(vacancies):
    """Парсинг и структурирование данных о вакансиях"""
    parsed_data = []

    print("\n🧠 Обработка данных...")

    for idx, vacancy in enumerate(vacancies, 1):
        # Основная информация
        salary = vacancy.get("salary") or {}

        # Детали вакансии
        details = get_vacancy_details(vacancy["id"]) if vacancy.get("id") else {}

        # Форматирование даты
        published_at = vacancy.get("published_at")
        if published_at:
            published_at = datetime.strptime(published_at, "%Y-%m-%dT%H:%M:%S%z").strftime("%Y-%m-%d %H:%M")

        # Извлечение адреса
        address = details.get("address") or {}
        address_str = ", ".join(filter(None, [
            address.get("city"),
            address.get("street"),
            address.get("building")
        ]))

        parsed_data.append({
            "id": vacancy.get("id"),
            "название": vacancy.get("name"),
            "компания": vacancy.get("employer", {}).get("name"),
            "зарплата_от": salary.get("from"),
            "зарплата_до": salary.get("to"),
            "валюта": salary.get("currency"),
            "опыт": vacancy.get("experience", {}).get("name"),
            "занятость": vacancy.get("employment", {}).get("name"),
            "график": vacancy.get("schedule", {}).get("name"),
            "ссылка": vacancy.get("alternate_url"),
            "дата_публикации": published_at,
            "описание": clean_html(details.get("description", "")),
            "навыки": ", ".join([s["name"] for s in details.get("key_skills", [])]),
            "регион": details.get("area", {}).get("name"),
            "адрес": address_str,
            "тип_компании": vacancy.get("employer", {}).get("type")
        })

        # Индикатор прогресса
        if idx % 50 == 0:
            print(f"  ➤ Обработано {idx}/{len(vacancies)} вакансий")

    return parsed_data

def get_vacancy_details(vacancy_id):
    """Получение детальной информации о вакансии"""
    try:
        response = requests.get(f"https://api.hh.ru/vacancies/{vacancy_id}")
        response.raise_for_status()
        return response.json()
    except:
        return {}

def create_vacancies_table(data):
    """Создаем DataFrame с вакансиями"""
    df = pd.DataFrame(data)

    # Оптимизация типов данных
    df["зарплата_от"] = pd.to_numeric(df["зарплата_от"], errors="coerce")
    df["зарплата_до"] = pd.to_numeric(df["зарплата_до"], errors="coerce")
    df["дата_публикации"] = pd.to_datetime(df["дата_публикации"], errors="coerce")

    # Удаление дубликатов
    initial_count = len(df)
    df = df.drop_duplicates(subset="id")
    final_count = len(df)

    if initial_count != final_count:
        print(f"🔁 Удалено {initial_count - final_count} дубликатов")

    return df

if __name__ == "__main__":
    # Параметры сбора
    KEYWORD = "Аналитик данных"  # Ключевое слово для поиска
    REGIONS = [113]  # 113 = Россия, 1 = Москва, 2 = СПб
    MAX_VACANCIES = 1000

    print("="*70)
    print(f"🚀 Запуск сбора вакансий по ключевому слову: '{KEYWORD}'")
    print("="*70)

    # Получаем вакансии
    raw_vacancies = get_all_vacancies(
        search_query=KEYWORD,
        region=REGIONS,
        max_vacancies=MAX_VACANCIES
    )

    # Парсим данные
    parsed_data = parse_vacancies(raw_vacancies)

    # Создаем таблицу
    df = create_vacancies_table(parsed_data)

    # Сохраняем в CSV
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    filename = f"hh_{KEYWORD.replace(' ', '_')}_{timestamp}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")

    print("\n" + "="*70)
    print(f"✅ Сбор завершен! Сохранено {len(df)} вакансий")
    print(f"💾 Файл: {filename}")
    print("="*70)

    # Аналитическая сводка
    if not df.empty:
        print("\n📊 Статистика:")
        print(f"- Средняя ЗП от: {df['зарплата_от'].mean():.0f} {df['валюта'].mode()[0]}")
        print(f"- Средняя ЗП до: {df['зарплата_до'].mean():.0f} {df['валюта'].mode()[0]}")
        print(f"- Популярные навыки: {', '.join(df['навыки'].str.split(', ').explode().value_counts().head(5).index.tolist())}")
        print(f"- Топ регионов: {', '.join(df['регион'].value_counts().head(3).index.tolist())}")

🚀 Запуск сбора вакансий по ключевому слову: 'Аналитик данных'
🔍 Поиск вакансий по запросу: 'Аналитик данных'
🌍 Регион: Россия (ID: 113)
  ➤ Страница 1: +100 вакансий (всего: 100)
  ➤ Страница 2: +100 вакансий (всего: 200)
  ➤ Страница 3: +100 вакансий (всего: 300)
  ➤ Страница 4: +100 вакансий (всего: 400)
  ➤ Страница 5: +100 вакансий (всего: 500)
  ➤ Страница 6: +100 вакансий (всего: 600)
  ➤ Страница 7: +33 вакансий (всего: 633)

🧠 Обработка данных...
  ➤ Обработано 50/633 вакансий
  ➤ Обработано 100/633 вакансий
  ➤ Обработано 150/633 вакансий
  ➤ Обработано 200/633 вакансий
  ➤ Обработано 250/633 вакансий
  ➤ Обработано 300/633 вакансий
  ➤ Обработано 350/633 вакансий
  ➤ Обработано 400/633 вакансий
  ➤ Обработано 450/633 вакансий
  ➤ Обработано 500/633 вакансий
  ➤ Обработано 550/633 вакансий
  ➤ Обработано 600/633 вакансий

✅ Сбор завершен! Сохранено 633 вакансий
💾 Файл: hh_Аналитик_данных_20250802_1901.csv

📊 Статистика:
- Средняя ЗП от: 97250 RUR
- Средняя ЗП до: 134270 RUR
- 